In [1]:
# import
import pandas as pd
import numpy as np
import altair as alt
import json
import pycountry_convert as pc

# enable Altair to work with data with >5k rows
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
# load data
goals = pd.read_csv('goals.csv')

In [3]:
# view data
goals.head()

,date,home_team,away_team,scorer_team,scorer,minute,own_goal,penalty,point_earned,home_goals,away_goals,winner,winner_team,penalty_point,home_penalties,away_penalties
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False,1,0,4,Uruguay,Away,0,0,0
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False,1,0,4,Uruguay,Away,0,0,0
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False,1,0,4,Uruguay,Away,0,0,0
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False,1,0,4,Uruguay,Away,0,0,0
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False,1,6,1,Argentina,Home,0,2,0


In [4]:
# group data by game
game_cols = ['date', 'home_team', 'away_team', 'home_goals', 'away_goals', 
        'winner', 'winner_team', 'home_penalties', 'away_penalties']

games = goals[game_cols].drop_duplicates()

games['year'] =  games['date'].astype('datetime64[ns]').dt.year 

games.reset_index(drop=True, inplace=True)

print(games.shape)
games.head()

(13986, 10)


,date,home_team,away_team,home_goals,away_goals,winner,winner_team,home_penalties,away_penalties,year
0,1916-07-02,Chile,Uruguay,0,4,Uruguay,Away,0,0,1916
1,1916-07-06,Argentina,Chile,6,1,Argentina,Home,2,0,1916
2,1916-07-08,Brazil,Chile,1,1,Tie,Tie,0,0,1916
3,1916-07-10,Argentina,Brazil,1,1,Tie,Tie,0,0,1916
4,1916-07-12,Brazil,Uruguay,1,2,Uruguay,Away,0,0,1916


In [5]:
# group data by team
team_cols = ['date', 'year', 'team', 'opponent', 'goals', 'opponent_goals', 
        'penalties_awarded', 'penalties_committed', 'won', 'home']

home_teams = games.copy()
home_teams = home_teams.rename(columns={'home_team': 'team',
                                        'away_team': 'opponent',
                                        'home_goals': 'goals',
                                        'away_goals': 'opponent_goals',
                                        'home_penalties': 'penalties_awarded',
                                        'away_penalties': 'penalties_committed'
                                       })
home_teams['won'] = np.where(home_teams['team'] == home_teams['winner'], 1, 0)
home_teams['home'] = 1
home_teams = home_teams[team_cols]

away_teams = games.copy()
away_teams = away_teams.rename(columns={'away_team': 'team',
                                        'home_team': 'opponent',
                                        'away_goals': 'goals',
                                        'home_goals': 'opponent_goals',
                                        'away_penalties': 'penalties_awarded',
                                        'home_penalties': 'penalties_committed'
                                       })
away_teams['won'] = np.where(away_teams['team'] == away_teams['winner'], 1, 0)
away_teams['home'] = 0
away_teams = away_teams[team_cols]

teams = pd.concat([home_teams, away_teams], axis=0)

teams.reset_index(drop=True, inplace=True)

# get continent from country
def get_continent(name):
    continents = {'SA': 'South America', 'EU': 'Europe', 'NA': 'North America', 
                  'AF': 'Africa', 'AS': 'Asia', 'OC': 'Oceania', None: None}
    try:
        code = pc.country_name_to_country_alpha2(name)
        cont = pc.country_alpha2_to_continent_code(code)
        return continents[cont]
    except: 
        pass

teams['team_continent'] = teams['team'].apply(get_continent)
teams['team_continent'] = np.where(teams['team'].isin(['Republic of Ireland','Northern Ireland','Wales', 
                                                     'Scotland','England', 'Kosovo']), 'Europe',
                          np.where(teams['team'].isin(['China PR', 'Timor-Leste']), 'Asia',
                          np.where(teams['team'].isin(['DR Congo']), 'Africa', 
                          np.where(teams['team'].isin(['Tahiti']), 'Oceania', teams['team_continent']))))

teams.head()

,date,year,team,opponent,goals,opponent_goals,penalties_awarded,penalties_committed,won,home,team_continent
0,1916-07-02,1916,Chile,Uruguay,0,4,0,0,0,1,South America
1,1916-07-06,1916,Argentina,Chile,6,1,2,0,1,1,South America
2,1916-07-08,1916,Brazil,Chile,1,1,0,0,0,1,South America
3,1916-07-10,1916,Argentina,Brazil,1,1,0,0,0,1,South America
4,1916-07-12,1916,Brazil,Uruguay,1,2,0,0,0,1,South America


In [6]:
# team summaries for use in world map
teams_sum = teams.copy()

# get earliest/latest year
teams_sum['min_year'] = teams_sum.groupby(['team'])['year'].transform('min')
teams_sum['latest_year'] = teams_sum.groupby(['team'])['year'].transform('max')

def bin_years(year):
    if year < 1940:
        return '1916-1940'
    elif year < 1960:
        return '1941-1960'
    elif year < 1980:
        return '1961-1980'
    elif year < 2000:
        return '1981-2000'
    else:
        return '2001-2023'
    
teams_sum['min_year_binned'] = teams_sum['min_year'].apply(bin_years)

# get count games
teams_sum['count_games'] = teams_sum.groupby(['team'])['team'].transform('count')


# add ISO 3166-1 numeric codes to teams data
iso = pd.read_csv('iso_3166.csv')
teams_sum = pd.merge(teams_sum, iso, left_on='team', right_on='name', how='left')

# get subset of columns
teams_sum.drop(columns=['date','year', 'opponent','goals','opponent_goals','penalties_awarded',
                        'penalties_committed','won','home','name'], axis=1, inplace=True)

# drop duplicates
teams_sum.drop_duplicates(inplace=True)

# reset index
teams_sum.reset_index(drop=True, inplace=True)

print(teams_sum.shape)
teams_sum.head()

(214, 7)


,team,team_continent,min_year,latest_year,min_year_binned,count_games,id
0,Chile,South America,1916,2022,1916-1940,365,152
1,Argentina,South America,1916,2022,1916-1940,419,32
2,Brazil,South America,1916,2022,1916-1940,440,76
3,Uruguay,South America,1916,2022,1916-1940,413,858
4,Paraguay,South America,1921,2022,1916-1940,346,600


In [7]:
# set custom theme
def custom_theme():
    return {
        'config': {
            'view': {
                'height': 400,
                'width': 400,
                
            },
            "axis": {
                "labelFontSize": 12, 
                "titleFontSize": 13,
            },
            "title": {
              "fontSize": 14,
              "subtitleFontSize": 13
            },
            "legend": {
              "labelFontSize": 12,
              "titleFontSize": 13
            },
            'mark': {
                'fill': '#005391'
            }
        }
    }

# register the custom theme under a chosen name
alt.themes.register('custom_theme', custom_theme)

# enable the newly registered theme
alt.themes.enable('custom_theme')

ThemeRegistry.enable('custom_theme')

#### Plot 1: Map of Countries

In [8]:
# world topo, edited to add Kosovo
world_topo = alt.topo_feature('world_topo.json', 'countries')

In [15]:
# plot 1
domain = ['1916-1940', '1941-1960', '1961-1980', '1981-2000', '2001-2023', 'No Data']
colors = ['#005391', '#0074cc', '#1a9cff', '#66bdff', '#b3deff', '#E0E0E0']

# world countries
teams1a = alt.Chart(world_topo).mark_geoshape(
        stroke='white', strokeWidth=0.5
    ).transform_filter(
        # remove Antarctica
        'datum.id!=10'
    ).project(
        type='naturalEarth1'
    ).transform_lookup(
        lookup='id', from_=alt.LookupData(data=teams_sum, key='id', fields=['min_year_binned'])
    ).encode(
        color = alt.condition('datum.min_year_binned != null', 'min_year_binned:N', alt.value('#E0E0E0'), 
                              scale=alt.Scale(domain=domain, range=colors),
                              title='Appearance Year')
    ).properties(
        height=350,
        width=750,
        title = "First Appearance in Men's International Football (1916-2023)"
    )

teams1a

alt.Chart(...)

#### Plot 2: Teams that Win Most by Year

In [23]:
domain = ['Africa', 'Asia', 'Europe', 'North America', 'South America', 'Multiple']
colors = ['#4c78a8', '#f58518', '#e45756', '#72b7b2', '#54a24b', '#b279a2']

base = alt.Chart(teams).mark_bar(
    ).transform_aggregate(
        # count of games and wins by team/year
        count_games='count(won)',
        count_wins='sum(won)',
        groupby=['team', 'year', 'team_continent']
    ).transform_window(
        # max game count by year
        max_games='max(count_games)',
        frame=[None, None],
        groupby=['year']
    ).transform_calculate(
        # mid-point for max games
        mid='datum.max_games/2',
        # win-rate
        prop_win='datum.count_wins/datum.count_games'
    ).transform_filter(
        # teams that played most games per year
        'datum.count_games >= datum.mid'
    ).transform_window(
        # team(s) with highest win-rate
        max_prop='max(prop_win)',
        frame=[None, None],
        groupby=['year']
    ).transform_filter(
        # filter out teams without highest win-rate
        'datum.prop_win == datum.max_prop'
    ).transform_joinaggregate(
        # max games for teams with highest win-rates
        count_games_max='max(count_games)',
        groupby=['year']
    ).transform_filter(
        # when there is a tie, take the team who played the most games
        'datum.count_games == datum.count_games_max'
    ).transform_calculate(
        # 1996 still has a tie
        team_continent_calc='datum.year != 1996 ? datum.team_continent : "Multiple"',
    ).encode(
        alt.Y('year:O', title='Year', 
              axis=alt.Axis(values=[1920, 1925, 1930, 1935, 1939, 1945, 1950, 1955, 1960, 1965,
                                    1970, 1975, 1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020])),
        alt.X('mean(max_prop):Q', title='Percent Games Won',
              axis=alt.Axis(format='%')),
        alt.Color('team_continent_calc:N', title='Team Continent', 
                  scale=alt.Scale(domain=domain, range=colors),
                  legend=alt.Legend(orient='right'))
    ).properties(
        height=500,
        width=200
    )

# split into two columns for better sizing
bars1 = base.transform_filter('datum.year <= 1974')
bars2 = base.transform_filter('datum.year > 1974').encode(alt.Y('year:O', title=None))

# add team names to chart
countries = base.mark_text(align='left', fontSize=11).transform_window(
        t = 'min(team)',
        t2 = 'max(team)',
        frame=[None, None],
        groupby=['year']
    ).transform_calculate(
        text = 'datum.t == datum.t2 ? datum.t : datum.t + "/" + datum.t2'
    ).encode(x=alt.value(202), text=alt.Text('text:N'))

countries1 = countries.transform_filter('datum.year <= 1974')
countries2 = countries.transform_filter('datum.year > 1974')

teams2 = alt.hconcat(bars1 + countries1, 
                     bars2 + countries2,  
                     title=alt.Title(
                           text='Teams with Highest Percent of Wins by Year',
                           subtitle='Including Only Teams that Played at Least Median Count Games by Year',
                           anchor='middle')
                    )
teams2

alt.HConcatChart(...)

#### Plot 3: Teams that Kick/Commit the Most Penalities

cut to 20, remove color encoding

In [26]:
base = alt.Chart(teams).mark_bar().transform_aggregate(
        mean_penalties_committed = 'mean(penalties_committed)',
        mean_penalties_awarded = 'mean(penalties_awarded)',
        tot_games = 'count(won)',
        tot_wins = 'sum(won)',
        groupby=['team']
    ).transform_calculate(
        win_percentage = 'datum.tot_wins / datum.tot_games'
    ).properties(width=200)

c1 = base.transform_window(
        rank='rank(mean_penalties_committed)',
        sort=[alt.SortField('mean_penalties_committed', order='descending')],
        frame = [None, None]
    ).transform_filter(
        'datum.rank <= 20'
    ).encode(
        alt.Y('team:N', title='Team',
              sort=alt.EncodingSortField(field='mean_penalties_committed', order='descending')),
        alt.X('mean_penalties_committed:Q', title='Mean Penalties Committed per Game', 
              scale=alt.Scale(domain=[0, 0.6]))
    )


c2 = base.transform_window(
        rank='rank(mean_penalties_awarded)',
        sort=[alt.SortField('mean_penalties_awarded', order='descending')],
        frame = [None, None]
    ).transform_filter(
        'datum.rank <= 20'
    ).encode(
        alt.Y('team:N', title=None,
              sort=alt.EncodingSortField(field='mean_penalties_awarded', order='descending')),
        alt.X('mean_penalties_awarded:Q', title='Mean Penalties Awarded per Game', 
              scale=alt.Scale(domain=[0, 0.6]))
    )

teams3 = alt.hconcat(c1,c2, 
                     title=alt.Title(
                           text='Top 20 Teams for Most Penalties Committed and Awarded',
                           subtitle='Normal Time Minutes Only',
                           anchor='middle')
                    )

teams3

alt.HConcatChart(...)

#### Plot 4: Highest Scoring Teams per Game

remove color encoding

In [53]:
colors = ['#4c78a8', '#f58518', '#e45756', '#72b7b2', '#eeca3b', '#54a24b']

base = alt.Chart(teams).mark_circle(opacity=0.2).transform_aggregate(
        mean_goals = 'mean(goals)',
        mean_opp_goals = 'mean(opponent_goals)',
        tot_games = 'count(won)',
        tot_wins = 'sum(won)',
        groupby=['team', 'team_continent']
    ).transform_calculate(
        win_percentage = 'datum.tot_wins / datum.tot_games'
    ).encode(
        alt.X('mean_goals:Q', title='Mean Team Goals', 
              scale=alt.Scale(domain=[0,3]),
              axis=alt.Axis(values=[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0])),
        alt.Y('mean_opp_goals:Q', title='Mean Opponent Goals')
    ).properties(title='Team and Opponent Goals per Game', width=600, height=500)

points = base.encode(
    alt.Size('win_percentage:Q', title='Win Percentage',
         legend=alt.Legend(format='%')))

# get teams often considered the best in the world
top10 = base.mark_point(color='black', fill=None, size=500).transform_filter(
    {'field': 'team', 'oneOf': ['Brazil', 'Germany', 'Italy', 'Argentina', 'England', 'Spain',
                               'France', 'Netherlands', 'Uruguay', 'Sweden']}
)

top10text_base = base.mark_text().encode(alt.Text('team:N'))

top10text = top10text_base.mark_text(align='right', baseline='top', dy=8, dx=-8, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Spain', 'Italy', 'Sweden']})
top10text += top10text_base.mark_text(align='right', baseline='bottom', dy=-6, dx=-8, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Uruguay', 'France']})
top10text += top10text_base.mark_text(align='left', baseline='bottom', dy=-8, dx=8, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Germany']})
top10text += top10text_base.mark_text(align='left', baseline='top', dy=10, dx=10, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['England']})
top10text += top10text_base.mark_text(align='right', baseline='bottom', dy=-14, dx=-2, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Argentina']})
top10text += top10text_base.mark_text(align='right', baseline='bottom', dy=-10, dx=-2, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Brazil']})
top10text += top10text_base.mark_text(align='left', baseline='top', dy=4, dx=12, fontSize=12
    ).transform_filter({'field': 'team', 'oneOf': ['Netherlands']})

# combine
teams4 = points + top10 + top10text

teams4

alt.LayerChart(...)